In [1]:
import json
import os
# Uncomment below when using MySQL
# import mysql.connector

In [ ]:
# --- File and (Optional) MySQL Functions ---

DATA_FILE = "accounts.json"

def load_data():
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, "r") as f:
            return json.load(f)
    return {}

def save_data(data):
    with open(DATA_FILE, "w") as f:
        json.dump(data, f, indent=2)

# MySQL connection (optional)
def get_connection():
    return mysql.connector.connect(
        host="localhost",
        user="your_user",
        password="your_pass",
        database="atm_db"
    )

In [ ]:
# Authentication
def authenticate(account_number, pin, role, data):
    # MySQL version (optional)
    # conn = get_connection()
    # cursor = conn.cursor(dictionary=True)
    # cursor.execute("SELECT * FROM accounts WHERE account_number=%s AND pin=%s", (account_number, pin))
    # account = cursor.fetchone()
    # cursor.close()
    # conn.close()
    # return account

    account = data.get(account_number)
    if account and account["pin"] == pin:
        return account
    return None

In [ ]:
# --- User Functions ---

def user_menu(account, data, account_number):
    while True:
        print("\n1) Check Balance  2) Deposit  3) Withdraw  4) Exit")
        choice = input("Choose an option: ")

        if choice == "1":
            print(f"Balance: ${account['balance']:.2f}")
        elif choice == "2":
            amt = float(input("Amount to deposit: "))
            account['balance'] += amt
            print("Deposit successful.")
        elif choice == "3":
            amt = float(input("Amount to withdraw: "))
            if amt <= account['balance']:
                account['balance'] -= amt
                print("Withdrawal successful.")
            else:
                print("Insufficient funds.")
        elif choice == "4":
            print("Goodbye!")
            break
        else:
            print("Invalid option.")
        data[account_number] = account
        save_data(data)


In [ ]:
# --- Admin Functions ---

def admin_menu(data):
    while True:
        print("\nAdmin Options: 1) Open Account  2) Close Account  3) Modify Account  4) Exit")
        choice = input("Choose an option: ")

        if choice == "1":
            acc = input("New account number: ")
            if acc in data:
                print("Account already exists.")
                continue
            name = input("Name: ")
            pin = input("PIN: ")
            data[acc] = {"name": name, "pin": pin, "balance": 0.0}
            print("Account created.")
        elif choice == "2":
            acc = input("Account to close: ")
            if acc in data:
                del data[acc]
                print("Account closed.")
            else:
                print("Account not found.")
        elif choice == "3":
            acc = input("Account to modify: ")
            if acc in data:
                name = input("New name: ")
                pin = input("New PIN: ")
                data[acc]["name"] = name
                data[acc]["pin"] = pin
                print("Account updated.")
            else:
                print("Account not found.")
        elif choice == "4":
            break
        else:
            print("Invalid option.")
        save_data(data)

In [ ]:
# --- Main Program ---

def main():
    print("Welcome to the ATM System")
    data = load_data()

    role = input("Login as 'user' or 'admin': ").lower().strip()
    acc = input("Account number: ")
    pin = input("PIN: ")

    account = authenticate(acc, pin, role, data)

    if not account:
        print("Login failed.")
        return

    print(f"Welcome, {account['name']}!")

    if role == "user":
        user_menu(account, data, acc)
    elif role == "admin":
        admin_menu(data)
    else:
        print("Unknown role.")

if __name__ == "__main__":
    main()
